# About

Proof of concept notebook for finetuning GPT using HuggingFace and Torch 

## Setup

In [1]:
# Download libraries for environment. 

import sys 
import os 

# Env. vars to check if the notebook is running on colab, kaggle etc. 
IS_COLAB = "google.colab" in sys.modules 
IS_KAGGLE = "kaggle_secrets" in sys.modules 
IS_LOCAL = not (IS_COLAB or IS_KAGGLE)

if IS_COLAB:
    # Install the packages 
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers
    %pip install -q -U datasets
    print("You can safely ignore the package incompatibility errors.")
    # Mount the drive 
    from google.colab import drive 
    drive.mount("/drive")

In [2]:

import os
import pandas as pd
import numpy as np
from tqdm import tqdm 

import random 
import shutil 
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# Pytorch imports 
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

# Others 
import glob 

# Transformers 
import transformers 
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import AutoTokenizer
import datasets 

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



In [10]:
# --  Set environment global vars. 

# Shared env. vars. 
GLOBAL_SEED = 42 
IS_CUDA_ENV = torch.cuda.is_available()
GLOBAL_DEVICE = torch.device('cuda') if IS_CUDA_ENV else torch.device('cpu')
SET_SEED = True # If true, sets the global seeds for this notebook. 

if IS_LOCAL:
    SMALL_MODEL = True if not IS_CUDA_ENV else False # Use a small dataset if no cuda env. 

if IS_COLAB:
    SMALL_MODEL = False 

In [11]:
# Configuring env. 
if SET_SEED:
    # to make this notebook's output stable across runs
    np.random.seed(GLOBAL_SEED) 
    torch.manual_seed(GLOBAL_SEED)

In [82]:
# Project Paths
NOTEBOOK_NAME = "gpt_finetune_poc"
PROJECT_ROOT_DIR = "/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue" 
# --- Input data dirs. 
DATASET_NAME = "daily_dialog_poc"
DATASET_TYPE = "csv"
PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT_DIR,"data", "processed", DATASET_NAME)

# --- Result dirs. 
SAVE_MODEL_DIR = os.path.join(PROJECT_ROOT_DIR,"models",NOTEBOOK_NAME)
REPORTS_DIR = os.path.join(PROJECT_ROOT_DIR,"reports",NOTEBOOK_NAME)

os.makedirs(REPORTS_DIR,exist_ok=True)
os.makedirs(SAVE_MODEL_DIR,exist_ok=True)


In [13]:
# Methods for GPU Support 

def to_device(data, device):
    """Move tensor(s) to the given device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x,device) for x in data] 
    return data.to(device, non_blocking=True) 

class DeviceDataLoader():
    """Wrapper for a dataloader to move all the data to the specified device"""
    def __init__(self, dl, device):
        self.dl = dl 
        self.device = device 

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)


## Finetuning HuggingFace GPT

In [77]:

# NOTE: The below should be the same in the dataset - assuming there are 2 speakers! 
SPEAKER_1_TOKEN = "<SP1>"
SPEAKER_2_TOKEN = "<SP2>"
CONV_START_TOKEN = "<START>"
CONV_END_TOKEN = "<END>"
PAD_TOKEN = "<PAD>"
EOS_TOKEN = "<|endoftext|>"



In [81]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(REPORTS_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


### Tokenizing

In [78]:
# Tokenizer vars. 

TOKENIZER_CHECKPOINT = "gpt2"
TOKENIZER_BATCH_SIZE = 128

In [17]:
# Load the tokenizer with special tokens defined. 
tokenizer = AutoTokenizer.from_pretrained(
    TOKENIZER_CHECKPOINT, 
    pad_token = PAD_TOKEN, 
    eos_token = EOS_TOKEN, 
    additional_special_tokens=(
        SPEAKER_1_TOKEN, SPEAKER_2_TOKEN, CONV_START_TOKEN, 
        CONV_END_TOKEN))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [67]:
# Save the tokenizer after adding new tokens 
tokenizer.save_pretrained(SAVE_MODEL_DIR) 

('/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc/tokenizer_config.json',
 '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc/special_tokens_map.json',
 '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc/vocab.json',
 '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc/merges.txt',
 '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc/added_tokens.json',
 '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc/tokenizer.json')

In [69]:
tokenizer = tokenizer.from_pretrained(SAVE_MODEL_DIR)
tokenizer

PreTrainedTokenizerFast(name_or_path='/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/models/gpt_finetune_poc', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<PAD>', 'additional_special_tokens': ['<SP1>', '<SP2>', '<START>', '<END>']})

In [18]:
# Create the data collator, which is responsible for creating batches from the
# datasets during training.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False, 
    return_tensors="pt")


In [29]:
dataset_paths = glob.glob("{}/*.csv".format(PROCESSED_DATA_DIR))
dataset_paths = {os.path.splitext(os.path.basename(p))[0] : p for p in dataset_paths}
# Only keep the required keys / verify that they exist. 
dataset_paths

{'validation': '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/data/processed/daily_dialog_poc/validation.csv',
 'test': '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/data/processed/daily_dialog_poc/test.csv',
 'train': '/Users/muhammadumair/Documents/Repositories/mumair01-repos/GPT-Monologue-to-Dialogue/data/processed/daily_dialog_poc/train.csv'}

In [31]:
# NOTE: The TextDataset is deprecated - please see the next cell for the updated 
# method of creating the text dataset. Note that this is still included in 
# case the new approach does not work. 

#####  UNCOMMENT IF NEEDED ######

# train_dataset = TextDataset(
#     tokenizer=tokenizer,
#     file_path=dataset_paths["train"],
#     block_size=128)
# validation_dataset = TextDataset(
#     tokenizer=tokenizer,
#     file_path=dataset_paths["validation"],
#     block_size=128)

##################################

In [32]:
from datasets import load_dataset

In [42]:
# Loading text dataset using new method 
dataset = load_dataset(DATASET_TYPE, data_files=dataset_paths)

Using custom data configuration default-8282c7f272c6bbd2
Reusing dataset csv (/Users/muhammadumair/.cache/huggingface/datasets/csv/default-8282c7f272c6bbd2/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/3 [00:00<?, ?it/s]

In [51]:
# Each row in this dataset is an utterance. 
dataset['train']

Dataset({
    features: ['Unnamed: 0', 'convID', 'Utterance'],
    num_rows: 36
})

In [52]:
dataset['train'][0]['Utterance']

'<START>'

In [59]:
# Once loaded, the dataset needs to be processed 
def tokenize_fn(tokenizer):
    return lambda data: tokenizer(data["Utterance"], truncation=True) 
tokenized_datasets = dataset.map(
    tokenize_fn(tokenizer), batched=True, batch_size=TOKENIZER_BATCH_SIZE)

  0%|          | 0/43 [00:00<?, ?ba/s]

  0%|          | 0/34 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

In [88]:
tokenized_datasets['train']

Dataset({
    features: ['Unnamed: 0', 'convID', 'Utterance', 'input_ids', 'attention_mask'],
    num_rows: 36
})

# Model and Training

### Model / Training

In [83]:
MODEL_CHECKPOINT = "distilgpt2" if SMALL_MODEL else "gpt2-large"


In [84]:
from transformers import AutoModelForCausalLM

In [85]:
# Load the model 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_CHECKPOINT, 
    pad_token_id = tokenizer.pad_token_id, 
    eos_token_id = tokenizer.eos_token_id
)
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [86]:
# Defining training arguments 
training_args = TrainingArguments(
        output_dir=SAVE_MODEL_DIR,
        overwrite_output_dir=False,
        num_train_epochs=1,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        eval_steps=1,
        save_steps=1,
        warmup_steps=1,
        prediction_loss_only=True,
        evaluation_strategy='epoch',
        logging_dir=REPORTS_DIR)



In [100]:
# Create the trainer 
# NOTE: Trainer should automatically put the model and dataset to GPU 
trainer = Trainer(
    model=model,
    args=training_args, 
    data_collator=data_collator, 
    train_dataset=tokenized_datasets['train'], 
    eval_dataset=tokenized_datasets['validation']
)

In [101]:
import gc 

In [102]:
# Clear caches before training 
torch.cuda.empty_cache()
gc.collect 


<function gc.collect(generation=2)>

In [103]:

trainer.train() 
trainer.save_model()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: Unnamed: 0, convID, Utterance. If Unnamed: 0, convID, Utterance are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/Users/muhammadumair/anaconda3/envs/gpt_proj/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 36
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3


  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 